
# Analisis datos de juegos competitivos de League Of Legends

Importando librerías y datos

In [162]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

data = pd.read_excel('/content/2024_LoL_esports_match_data_from_OraclesElixir.xlsx')
data

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12271,LOLTMNT01_55111,complete,NaN,LCO,2024,Split 1,0,2024-02-13 10:28:54,2,14.02,...,81.0,2087.0,2145.0,61.0,2.0,3.0,2.0,2.0,1.0,4.0
12272,LOLTMNT01_55111,complete,NaN,LCO,2024,Split 1,0,2024-02-13 10:28:54,2,14.02,...,106.0,-417.0,-824.0,15.0,2.0,2.0,1.0,4.0,2.0,2.0
12273,LOLTMNT01_55111,complete,NaN,LCO,2024,Split 1,0,2024-02-13 10:28:54,2,14.02,...,14.0,543.0,613.0,4.0,1.0,5.0,2.0,1.0,5.0,3.0
12274,LOLTMNT01_55111,complete,NaN,LCO,2024,Split 1,0,2024-02-13 10:28:54,2,14.02,...,525.0,-4354.0,-3626.0,-114.0,7.0,13.0,10.0,10.0,12.0,7.0


# ¿Cuántos equipos han participado?

In [4]:
#¿cuántos equipos participaron?
x = data.teamname.unique()
y = len(x)
y

253

# ¿Influye el lado del mapa en el resultado?

In [183]:
side_filt = data[data['result'] == 1]
result_and_side = side_filt[side_filt['position'] == 'team']
side = pd.DataFrame(result_and_side.value_counts(['side', 'result'])).rename(columns={0:'Count'}).sort_values('side')
side

,,Count
side,result,
Blue,1,558
Red,1,465


In [184]:
# Filtrar los datos donde el resultado es 1
side_filt = data.loc[data['result'] == 1]

# Contar los valores de 'side' y 'result'
side = pd.DataFrame(side_filt.value_counts(['side', 'result'])).rename(columns={0:'Count'}).sort_values('side').reset_index()

fig = px.pie(side, values='Count', names='side', title='Porcentaje partidas ganadas dependiendo lado del mapa')

fig.show()

In [185]:
Liga = pd.DataFrame(data['league'].value_counts()).reset_index()
Liga.columns = ['League', 'Count']

fig = px.bar(Liga, x='League', y='Count', title='Partidas jugadas en cada Liga')
fig.update_layout(
    yaxis_title = 'Partidas',
    xaxis_title = 'Nombre de la Liga')

fig.show()

In [186]:

Liga = pd.DataFrame(data['league'].value_counts(normalize=True) * 100).reset_index()
Liga.columns = ['League', 'Percentage']

fig = px.pie(Liga, values='Percentage', names='League', title='Porcentaje de cada liga')

fig.show()

# ¿Influye primera sangre para ganar?

In [187]:
# Filtrar todas las filas donde 'firstblood' es igual a 1
data_filtered = data[data['result'] == 1]
por_team = data_filtered[data_filtered['position'] == 'team']

# Calcular el total de filas donde 'firstblood' es igual a 1
total_victoria = len(por_team)

# Filtrar las filas donde 'result' es igual a 1 y 'firstblood' es igual a 1
result_and_first_blood = por_team[por_team['firstblood'] == 1]

# Calcular el total de filas donde 'result' es igual a 1 y 'firstblood' es igual a 1
total_result_and_first_blood = len(result_and_first_blood)

# Calcular el porcentaje
percentage = (total_result_and_first_blood / total_victoria) * 100


print('El porcentaje de ganar después de haber obtenido primera sangre:', percentage)

El porcentaje de ganar después de haber obtenido primera sangre: 60.70381231671554


In [179]:
porcentaje_restante = 100 - percentage

# Crear un DataFrame con los porcentajes
df = pd.DataFrame({'Porcentaje': [percentage, porcentaje_restante],
                   'Estado': ['Porcentaje de ganar', 'Porcentaje de derrota']})

fig = px.pie(df, values='Porcentaje', names='Estado', title='Porcentajes de resultado teniendo Primera Sangre')

fig.show()

In [188]:
print('El total de partidas ganadas es: ',total_victoria)

El total de partidas ganadas es:  1023


In [193]:
print('Del total de partidas ganadas,',total_result_and_first_blood, ' de ellas ganó el equipo que se llevó la first blood')

Del total de partidas ganadas, 621  de ellas ganó el equipo que se llevó la first blood


# Promedio de Farmeo

In [152]:
# Obtener todos los valores únicos en la columna 'position'
posiciones_unicas = data['position'].unique()

# Crear un diccionario para almacenar los promedios de 'minionkills' para cada posición
promedios_por_posicion = {}

# Calcular el promedio de 'minionkills' para cada posición única
for posicion in posiciones_unicas:
    # Filtrar las filas donde 'position' es igual a la posición actual
    data_posicion = data[data['position'] == posicion]
    # Calcular el promedio de 'minionkills' para esta posición
    promedio = data_posicion['minionkills'].mean()
    promedio = round(promedio)
    # Almacenar el promedio en el diccionario
    promedios_por_posicion[posicion] = promedio


# Mostrar los promedios por posición
for posicion, promedio in promedios_por_posicion.items():
    print(f"Promedio de minion kills para '{posicion}': {promedio}")

Promedio de minion kills para 'top': 251
Promedio de minion kills para 'jng': 25
Promedio de minion kills para 'mid': 273
Promedio de minion kills para 'bot': 263
Promedio de minion kills para 'sup': 49
Promedio de minion kills para 'team': 856


# Promedio duración de Juego

In [194]:
# Filtrar los datos donde 'position' contiene el valor 'team'
data_filt = data[data['position'].str.contains('team', case=False)]

# Calcular el promedio de los valores de 'gamelength' para los datos filtrados
promedio_gamelength = data_filt['gamelength'].mean()
promedio_gamelength = round(promedio_gamelength)

# Mostrar el resultado del promedio
print("El promedio de duración del juego es de", promedio_gamelength/60, 'minutos')


El promedio de duración del juego es de 32.2 minutos


# ¿Influye la visión en el mapa?

In [195]:
# Filtrar los datos donde 'position' es igual a 'team' y 'result' es igual a 1
data_vision = data[(data['position'] == 'team') & (data['result'] == 1)]

# Calcular el promedio de la columna 'visionscore' para los datos filtrados
promedio_visionscore = data_vision['visionscore'].mean()
promedio_visionscore = round(promedio_visionscore)

# Mostrar el resultado del promedio
print("El promedio de visionscore para 'position' = 'team' y 'result' = 1 es:", promedio_visionscore)

El promedio de visionscore para 'position' = 'team' y 'result' = 1 es: 283


In [161]:
# Filtrar los datos donde 'position' es igual a 'team' y 'result' es igual a 1
data_filtered1 = data[(data['position'] == 'team') & (data['result'] == 0)]

# Calcular el promedio de la columna 'visionscore' para los datos filtrados
promedio_visionscore1 = data_filtered1['visionscore'].mean()
promedio_visionscore1 = round(promedio_visionscore1)

# Mostrar el resultado del promedio
print("El promedio de visionscore para 'position' = 'team' y 'result' = 1 es:", promedio_visionscore1)

El promedio de visionscore para 'position' = 'team' y 'result' = 1 es: 255
